In [1]:
import pandas as pd 
# Read the CSV file into a DataFrame
df = pd.read_csv('remove_9.csv')

In [1]:
import pandas as pd 
# Read the CSV file into a DataFrame
df = pd.read_csv('remove_9.csv')

df.shape

(2758, 15)

select feature based on average they appear

SQFT	6
WGTADJ	6
LOCATION	5
REGION	5

SECTIONS	4
BEDROOMS	3
WEIGHT	3


FOOTINGS	2
SECURED	2
STATUS	2
TITLED	1

In [2]:
# List all columns in the DataFrame
print(df.columns.tolist())

['WEIGHT', 'STATUS', 'FINALDEST', 'FOOTINGS', 'LEASE', 'LOCATION', 'REGION', 'PIERS', 'SECURED', 'TITLED', 'SECTIONS', 'PRICE', 'SQFT', 'BEDROOMS', 'WGTADJ']


drop these features 
['WEIGHT', 'STATUS', 'FINALDEST', 'FOOTINGS', 'LEASE', '', '', 'PIERS', 'SECURED', 'TITLED', 'SECTIONS', '', '', 'BEDROOMS', '']

In [3]:
# Delete multiple columns
columns_to_drop = ['WEIGHT', 'STATUS', 'FINALDEST', 'FOOTINGS', 'LEASE', 'PIERS', 'SECURED', 'TITLED', 'SECTIONS', 'BEDROOMS']
df = df.drop(columns_to_drop, axis=1)

# Print the DataFrame
print(df)

      LOCATION  REGION   PRICE  SQFT  WGTADJ
0            1       3   90000  1200  0.9369
1            2       3   82000  1200  0.9369
2            3       3   68000  1100  0.9369
3            2       4   90000  1200  0.9369
4            1       4   90000  1200  0.9369
...        ...     ...     ...   ...     ...
2753         2       4  140000  1400  1.0439
2754         1       1  150000  1600  1.0439
2755         1       1  150000  1600  1.0439
2756         1       1  220000  1500  1.0439
2757         3       2  110000  1100  1.0439

[2758 rows x 5 columns]


In [4]:
# Save the modified DataFrame to a new CSV file
df.to_csv('feature_selection2.csv', index=False)

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Read the CSV file into a DataFrame
df = pd.read_csv('feature_selection2.csv')

# Separate the target variable
target_variable = 'PRICE'
y = df[target_variable]
X = df.drop(target_variable, axis=1)

# Create a scaler object
scaler = StandardScaler()

# Fit the scaler to the features (excluding the target variable)
scaler.fit(X)

# Transform the features using the scaler
X_scaled = scaler.transform(X)

# Convert the scaled features back into a DataFrame
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Optional: Combine the scaled features with the target variable
df_scaled = pd.concat([X_scaled, y], axis=1)

# Print or save the scaled DataFrame
print(df_scaled)  # Or use 
df_scaled.to_csv('scaled_features2.csv', index=False) 


      LOCATION    REGION      SQFT    WGTADJ   PRICE
0    -0.969025  0.304856 -0.643077 -2.458163   90000
1     0.158166  0.304856 -0.643077 -2.458163   82000
2     1.285358  0.304856 -0.870927 -2.458163   68000
3     0.158166  1.232883 -0.643077 -2.458163   90000
4    -0.969025  1.232883 -0.643077 -2.458163   90000
...        ...       ...       ...       ...     ...
2753  0.158166  1.232883 -0.187377  0.241780  140000
2754 -0.969025 -1.551198  0.268323  0.241780  150000
2755 -0.969025 -1.551198  0.268323  0.241780  150000
2756 -0.969025 -1.551198  0.040473  0.241780  220000
2757  1.285358 -0.623171 -0.870927  0.241780  110000

[2758 rows x 5 columns]


In [6]:
import pandas as pd
# Read the CSV file into a DataFrame
df = pd.read_csv('scaled_features2.csv')

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


# Assuming 'PRICE' is the target variable
target_variable = 'PRICE'

# Extract features and target variable
X = df.drop(target_variable, axis=1)
y = df[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Lasso Regressor
lasso_regressor = Lasso(alpha=0.01)  # You may need to tune the alpha parameter

# Fit the Lasso model
lasso_regressor.fit(X_train, y_train)

# Predict on the test set using Lasso
lasso_pred = lasso_regressor.predict(X_test)

# Initialize the XGBoost Regressor
xgb_regressor = XGBRegressor(n_estimators=100, random_state=42)

# Fit the XGBoost model
xgb_regressor.fit(X_train, y_train)

# Predict on the test set using XGBoost
xgb_pred = xgb_regressor.predict(X_test)

# Set the weights for the hybrid model
lasso_weight = 0.65
xgb_weight = 0.35

# Combine predictions with weights
hybrid_pred = lasso_weight * lasso_pred + xgb_weight * xgb_pred

# Calculate metrics for the hybrid model
rmse = np.sqrt(mean_squared_error(y_test, hybrid_pred))
rae = mean_absolute_error(y_test, hybrid_pred)
mape = np.mean(np.abs((y_test - hybrid_pred) / y_test)) * 100
rmsle = np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(hybrid_pred), 2)))

# Print the results
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))
print("Relative Absolute Error (RAE): {:.2f}".format(rae))
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))
print("Root Mean Squared Logarithmic Error (RMSLE): {:.2f}".format(rmsle))


Root Mean Squared Error (RMSE): 35327.71
Relative Absolute Error (RAE): 23734.47
Mean Absolute Percentage Error (MAPE): 17.78%
Root Mean Squared Logarithmic Error (RMSLE): 0.23


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


# Assuming 'PRICE' is the target variable
target_variable = 'PRICE'

# Extract features and target variable
X = df.drop(target_variable, axis=1)
y = df[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.45, random_state=42)

# Initialize the Lasso Regressor
lasso_regressor = Lasso(alpha=0.01)  # You may need to tune the alpha parameter

# Fit the Lasso model
lasso_regressor.fit(X_train, y_train)

# Predict on the test set using Lasso
lasso_pred = lasso_regressor.predict(X_test)

# Initialize the XGBoost Regressor
xgb_regressor = XGBRegressor(n_estimators=100, random_state=42)

# Fit the XGBoost model
xgb_regressor.fit(X_train, y_train)

# Predict on the test set using XGBoost
xgb_pred = xgb_regressor.predict(X_test)

# Set the weights for the hybrid model
lasso_weight = 0.65
xgb_weight = 0.35

# Combine predictions with weights
hybrid_pred = lasso_weight * lasso_pred + xgb_weight * xgb_pred

# Calculate metrics for the hybrid model
rmse = np.sqrt(mean_squared_error(y_test, hybrid_pred))
rae = mean_absolute_error(y_test, hybrid_pred)
mape = np.mean(np.abs((y_test - hybrid_pred) / y_test)) * 100
rmsle = np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(hybrid_pred), 2)))

# Print the results
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))
print("Relative Absolute Error (RAE): {:.2f}".format(rae))
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))
print("Root Mean Squared Logarithmic Error (RMSLE): {:.2f}".format(rmsle))


Root Mean Squared Error (RMSE): 35288.55
Relative Absolute Error (RAE): 23506.73
Mean Absolute Percentage Error (MAPE): 17.04%
Root Mean Squared Logarithmic Error (RMSLE): 0.23


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


# Assuming 'PRICE' is the target variable
target_variable = 'PRICE'

# Extract features and target variable
X = df.drop(target_variable, axis=1)
y = df[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42)

# Initialize the Lasso Regressor
lasso_regressor = Lasso(alpha=0.01)  # You may need to tune the alpha parameter

# Fit the Lasso model
lasso_regressor.fit(X_train, y_train)

# Predict on the test set using Lasso
lasso_pred = lasso_regressor.predict(X_test)

# Initialize the XGBoost Regressor
xgb_regressor = XGBRegressor(n_estimators=100, random_state=42)

# Fit the XGBoost model
xgb_regressor.fit(X_train, y_train)

# Predict on the test set using XGBoost
xgb_pred = xgb_regressor.predict(X_test)

# Set the weights for the hybrid model
lasso_weight = 0.65
xgb_weight = 0.35

# Combine predictions with weights
hybrid_pred = lasso_weight * lasso_pred + xgb_weight * xgb_pred

# Calculate metrics for the hybrid model
rmse = np.sqrt(mean_squared_error(y_test, hybrid_pred))
rae = mean_absolute_error(y_test, hybrid_pred)
mape = np.mean(np.abs((y_test - hybrid_pred) / y_test)) * 100
rmsle = np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(hybrid_pred), 2)))

# Print the results
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))
print("Relative Absolute Error (RAE): {:.2f}".format(rae))
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))
print("Root Mean Squared Logarithmic Error (RMSLE): {:.2f}".format(rmsle))


Root Mean Squared Error (RMSE): 34451.14
Relative Absolute Error (RAE): 23391.73
Mean Absolute Percentage Error (MAPE): 17.31%
Root Mean Squared Logarithmic Error (RMSLE): 0.23


linear regression 

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Read the CSV file
df = pd.read_csv('scaled_features2.csv')

# Assuming 'PRICE' is the target variable
target_variable = 'PRICE'

# Extract features and target variable
X = df.drop(target_variable, axis=1)
y = df[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
linear_regressor = LinearRegression()

# Fit the model
linear_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = linear_regressor.predict(X_test)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
rmsle = np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))

# Print the results
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))
print("Relative Absolute Error (RAE): {:.2f}".format(rae))
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))
print("Root Mean Squared Logarithmic Error (RMSLE): {:.2f}".format(rmsle))


Root Mean Squared Error (RMSE): 37549.51
Relative Absolute Error (RAE): 25864.00
Mean Absolute Percentage Error (MAPE): 19.61%
Root Mean Squared Logarithmic Error (RMSLE): 0.25


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Read the CSV file
df = pd.read_csv('scaled_features2.csv')

# Assuming 'PRICE' is the target variable
target_variable = 'PRICE'

# Extract features and target variable
X = df.drop(target_variable, axis=1)
y = df[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Ridge Regression model
ridge_regressor = Ridge(alpha=1.0)  # Set the regularization strength (alpha)

# Fit the model
linear_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = linear_regressor.predict(X_test)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
rmsle = np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))

# Print the results
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))
print("Relative Absolute Error (RAE): {:.2f}".format(rae))
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))
print("Root Mean Squared Logarithmic Error (RMSLE): {:.2f}".format(rmsle))



Root Mean Squared Error (RMSE): 37549.51
Relative Absolute Error (RAE): 25864.00
Mean Absolute Percentage Error (MAPE): 19.61%
Root Mean Squared Logarithmic Error (RMSLE): 0.25


lasso 

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Read the CSV file
df = pd.read_csv('scaled_features2.csv')

# Assuming 'PRICE' is the target variable
target_variable = 'PRICE'

# Extract features and target variable
X = df.drop(target_variable, axis=1)
y = df[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize the LASSO Regression model
lasso_regressor = Lasso(alpha=0.1)  # Set the regularization strength (alpha)

# Fit the model
linear_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = linear_regressor.predict(X_test)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
rmsle = np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))

# Print the results
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))
print("Relative Absolute Error (RAE): {:.2f}".format(rae))
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))
print("Root Mean Squared Logarithmic Error (RMSLE): {:.2f}".format(rmsle))

Root Mean Squared Error (RMSE): 37549.51
Relative Absolute Error (RAE): 25864.00
Mean Absolute Percentage Error (MAPE): 19.61%
Root Mean Squared Logarithmic Error (RMSLE): 0.25


Lasso

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Read the CSV file
df = pd.read_csv('scaled_features2.csv')

# Assuming 'PRICE' is the target variable
target_variable = 'PRICE'

# Extract features and target variable
X = df.drop(target_variable, axis=1)
y = df[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the LASSO Regression model
lasso_regressor = Lasso(alpha=0.1)  # Set the regularization strength (alpha)

# Fit the LASSO model
lasso_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = lasso_regressor.predict(X_test)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
rmsle = np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))

# Print the results
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))
print("Relative Absolute Error (RAE): {:.2f}".format(rae))
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))
print("Root Mean Squared Logarithmic Error (RMSLE): {:.2f}".format(rmsle))


Root Mean Squared Error (RMSE): 37549.52
Relative Absolute Error (RAE): 25864.00
Mean Absolute Percentage Error (MAPE): 19.61%
Root Mean Squared Logarithmic Error (RMSLE): 0.25


Ridge

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Read the CSV file
df = pd.read_csv('scaled_features2.csv')

# Assuming 'PRICE' is the target variable
target_variable = 'PRICE'

# Extract features and target variable
X = df.drop(target_variable, axis=1)
y = df[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Ridge Regression model
ridge_regressor = Ridge(alpha=1.0)  # Set the regularization strength (alpha)

# Fit the Ridge model
ridge_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = ridge_regressor.predict(X_test)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
rmsle = np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))

# Print the results
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))
print("Relative Absolute Error (RAE): {:.2f}".format(rae))
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))
print("Root Mean Squared Logarithmic Error (RMSLE): {:.2f}".format(rmsle))

Root Mean Squared Error (RMSE): 37549.74
Relative Absolute Error (RAE): 25864.81
Mean Absolute Percentage Error (MAPE): 19.61%
Root Mean Squared Logarithmic Error (RMSLE): 0.25
